# Embeddings

This notebook goes over how to use the Embedding class in LangChain.

The Embedding class is a class designed for interfacing with embeddings. There are lots of Embedding providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embedding class in LangChain exposes two methods: `embed_documents` and `embed_query`. The largest difference is that these two methods have different interfaces: one works over multiple documents, while the other works over a single document. Besides this, another reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

## OpenAI

Let's load the OpenAI Embedding class.

In [1]:
from langchain.embeddings import OpenAIEmbeddings

In [2]:
embeddings = OpenAIEmbeddings()

In [3]:
text = "This is a test document."

In [4]:
query_result = embeddings.embed_query(text)

In [5]:
doc_result = embeddings.embed_documents([text])

## Cohere

Let's load the Cohere Embedding class.

In [1]:
from langchain.embeddings import CohereEmbeddings

In [2]:
embeddings = CohereEmbeddings(cohere_api_key= cohere_api_key)

In [3]:
text = "This is a test document."

In [4]:
query_result = embeddings.embed_query(text)

In [5]:
doc_result = embeddings.embed_documents([text])

## Hugging Face Hub
Let's load the Hugging Face Embedding class.

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
embeddings = HuggingFaceEmbeddings()

In [12]:
text = "This is a test document."

In [13]:
query_result = embeddings.embed_query(text)

In [14]:
doc_result = embeddings.embed_documents([text])

## TensorflowHub
Let's load the TensorflowHub Embedding class.

In [1]:
from langchain.embeddings import TensorflowHubEmbeddings

In [5]:
embeddings = TensorflowHubEmbeddings()

2023-01-30 23:53:01.652176: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 23:53:34.362802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
text = "This is a test document."

In [7]:
query_result = embeddings.embed_query(text)

## InstructEmbeddings
Let's load the HuggingFace instruct Embeddings class.

In [8]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [9]:
embeddings = HuggingFaceInstructEmbeddings(query_instruction="Represent the query for retrieval: ")

load INSTRUCTOR_Transformer
max_seq_length  512


In [10]:
text = "This is a test document."

In [11]:
query_result = embeddings.embed_query(text)

## Self Hosted Embeddings
Let's load the SelfHostedEmbeddings, SelfHostedHuggingFaceEmbeddings, and SelfHostedHuggingFaceInstructEmbeddings classes.

In [ ]:
from langchain.embeddings import (
    SelfHostedEmbeddings, 
    SelfHostedHuggingFaceEmbeddings, 
    SelfHostedHuggingFaceInstructEmbeddings
)
import runhouse as rh

In [ ]:
# For an on-demand A100 with GCP, Azure, or Lambda
gpu = rh.cluster(name="rh-a10x", instance_type="A100:1", use_spot=False)

# For an on-demand A10G with AWS (no single A100s on AWS)
# gpu = rh.cluster(name='rh-a10x', instance_type='g5.2xlarge', provider='aws')

# For an existing cluster
# gpu = rh.cluster(ips=['<ip of the cluster>'], 
#                  ssh_creds={'ssh_user': '...', 'ssh_private_key':'<path_to_key>'},
#                  name='my-cluster')

In [ ]:
embeddings = SelfHostedHuggingFaceEmbeddings(hardware=gpu)

In [6]:
text = "This is a test document."

In [ ]:
query_result = embeddings.embed_query(text)

And similarly for SelfHostedHuggingFaceInstructEmbeddings:

In [ ]:
embeddings = SelfHostedHuggingFaceInstructEmbeddings(hardware=gpu)

Now let's load an embedding model with a custom load function:

In [12]:
def get_pipeline():
    from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline  # Must be inside the function in notebooks
    model_id = "facebook/bart-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)
    return pipeline("feature-extraction", model=model, tokenizer=tokenizer)

def inference_fn(pipeline, prompt):
    # Return last hidden state of the model
    if isinstance(prompt, list):
        return [emb[0][-1] for emb in pipeline(prompt)] 
    return pipeline(prompt)[0][-1]

In [ ]:
embeddings = SelfHostedEmbeddings(
    model_load_fn=get_pipeline, 
    hardware=gpu,
    model_reqs=["./", "torch", "transformers"],
    inference_fn=inference_fn
)

In [ ]:
query_result = embeddings.embed_query(text)